<a href="https://colab.research.google.com/github/it21250156/DL-Lab03/blob/main/Face_Recognition_for_the_Happy_House_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# %% [code]
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
!unzip weights.zip

Archive:  weights.zip
   creating: weights/
  inflating: weights/bn1_b.csv       
  inflating: weights/bn1_m.csv       
  inflating: weights/bn1_v.csv       
  inflating: weights/bn1_w.csv       
  inflating: weights/bn2_b.csv       
  inflating: weights/bn2_m.csv       
  inflating: weights/bn2_v.csv       
  inflating: weights/bn2_w.csv       
  inflating: weights/bn3_b.csv       
  inflating: weights/bn3_m.csv       
  inflating: weights/bn3_v.csv       
  inflating: weights/bn3_w.csv       
  inflating: weights/conv1_b.csv     
  inflating: weights/conv1_w.csv     
  inflating: weights/conv2_b.csv     
  inflating: weights/conv2_w.csv     
  inflating: weights/conv3_b.csv     
  inflating: weights/conv3_w.csv     
  inflating: weights/dense_b.csv     
  inflating: weights/dense_w.csv     
  inflating: weights/inception_3a_1x1_bn_b.csv  
  inflating: weights/inception_3a_1x1_bn_m.csv  
  inflating: weights/inception_3a_1x1_bn_v.csv  
  inflating: weights/inception_3a_1x1_bn_w.csv  


In [5]:
# %% [code]
import fr_utils

In [6]:
# %% [code]

from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization,MaxPooling2D, AveragePooling2D
# from keras.layers.merge import Concatenate
from tensorflow.keras.layers import concatenate
from keras.layers import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
# from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer, InputSpec
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

import tensorflow
print(tensorflow.__version__)

2.17.0


In [7]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96), output_shape=(3,128))


In [8]:
# %% [code]
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [9]:
# %% [code]
# GRADED FUNCTION: triplet_loss

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)

    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)

    Returns:
    loss -- real number, value of the loss
    """

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)))
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)))
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.maximum(tf.reduce_mean(basic_loss), 0.0)
    ### END CODE HERE ###

    return loss


In [10]:
# %% [code]
# with tf.Session() as test:
with tf.compat.v1.Session() as test:
#    tf.set_random_seed(1)
    tf.random.set_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random.normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random.normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)

    print("loss = " + str(loss.eval()))



loss = 350.02734


In [11]:
# %% [code]
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)


In [12]:
# %% [code]
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)


In [13]:
# %% [code]
# GRADED FUNCTION: verify

def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".

    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be a resident of the Happy house.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras

    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """

    ### START CODE HERE ###

    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)

    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding-database[identity])

    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False

    ### END CODE HERE ###

    return dist, door_open

In [14]:
# %% [code]
verify("images/camera_0.jpg", "younes", database, FRmodel)

It's younes, welcome home!


(0.6671399, True)

In [16]:
# %% [code]
verify("images/camera_2.jpg", "kian", database, FRmodel)

It's not kian, please go away


(0.85868883, False)

In [17]:
# %% [code]
# GRADED FUNCTION: who_is_it

def who_is_it(image_path, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.

    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras

    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """

    ### START CODE HERE ###

    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(image_path, model)

    ## Step 2: Find the closest encoding ##

    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100

    #save L2 distances to interpret results
    output_data = {}

    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():

        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding-db_enc)

        #save L2 distances to interpret results
        output_data[f'{name}'] = dist

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist < min_dist:
            min_dist = dist
            identity = name

    ### END CODE HERE ###

    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))

    return min_dist, identity, output_data

In [18]:
# %% [code]
output = who_is_it("images/camera_0.jpg", database, FRmodel)

it's younes, the distance is 0.6671399


In [19]:
# %% [code]
#dictionary contains the L2 distance between target image encoding and database embeddings of other images
output[2]

{'danielle': 1.2107376,
 'younes': 0.6671399,
 'tian': 1.221301,
 'andrew': 0.99589044,
 'kian': 0.8566896,
 'dan': 0.7684447,
 'sebastiano': 0.854448,
 'bertrand': 0.7690577,
 'kevin': 0.8715994,
 'felix': 1.2063178,
 'benoit': 0.8959498,
 'arnaud': 0.8346038}

In [22]:
# %% [code]
from PIL import Image
img = Image.open("images/ronaldo1.jpg")
img = img.resize((96,96), Image.ANTIALIAS)
img.save("images/ronaldo1.jpg")

img = Image.open("images/ronaldo2.jpg")
img = img.resize((96,96), Image.ANTIALIAS)
img.save("images/ronaldo2.jpg")

database["ronaldo"] = img_to_encoding("images/ronaldo1.jpg", FRmodel)


<ipython-input-22-b183c19872e6>:4: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((96,96), Image.ANTIALIAS)
<ipython-input-22-b183c19872e6>:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((96,96), Image.ANTIALIAS)


In [23]:
# %% [code]
verify("images/camera_0.jpg", "ronaldo", database, FRmodel)

It's not ronaldo, please go away


(0.9850233, False)

In [24]:
# %% [code]
output = who_is_it("images/ronaldo2.jpg", database, FRmodel)

it's danielle, the distance is 0.5357453


In [25]:
# %% [code]
output[2]

{'danielle': 0.5357453,
 'younes': 1.302147,
 'tian': 0.54648215,
 'andrew': 0.9137047,
 'kian': 1.2934924,
 'dan': 1.2845762,
 'sebastiano': 1.1136677,
 'bertrand': 1.3282127,
 'kevin': 1.3130096,
 'felix': 1.2125703,
 'benoit': 0.8989191,
 'arnaud': 1.3376323,
 'ronaldo': 1.2037222}